### Challenge
* A challenge faced by many organizations today is that data is spread across the enterprise between many different systems. This includes data for reporting, dashboards, KPIs, and analytics.
* As data volumes grow it becomes more difficult to collect and analyze this data across all the different sources using existing tools.
* Data Lakes built using cloud storage provide scalable and cost-effective storage but accessing that data becomes challenging due to issues with performance,  reliability, and consistency. 

### Solution: Build a Lakehouse on Google Cloud Storage using Delta Lake
* Store all data in one location for all personas in your organization - Data Engineering, Data Science, Analysts. 
* Brings Scale, Performance, and Reliability to cloud data lakes. 
* Open-source project and no lock-in
* Learn more about Delta Lake: https://docs.gcp.databricks.com/delta/index.html

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<img src = "https://storage.googleapis.com/databricks-public-images/bmathew/delta_lake_lab_2.png" height=700 width=700>

- Please read the text in each of the notebook cells. 
- There are instructions in almost all of the cells that require input from you so that the code will run as expected.

### Create a Database for you to work in
- The database needs to be unique 
- Choose the username assigned to you for this class
- In the cell below, so that everyone has their own unique database, replace the - Replace the parameter "your-username" with your unique username
   - For example: odl_instructor_490106@databrickslabs.com --> odl_instructor_490106

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS odl_user_768559

- Create a directory where we will write data files to
- Replace the parameter "your-username" with your unique username

In [0]:
%fs
mkdirs /tmp/odl_user_768559/delta/clickstream

res0: Boolean = true

### Create a Delta table
- Make this an external/unmanaged table by specifying the location. By doing this, if we drop the table, then the data still remains.
- If we don't specify the location, then this becomes a managed table with the default location at /user/hive/warehouse/. If we drop the table, then the directory and all data will  be removed. 
- Delta will automatically compress the Parquet files for this table using Snappy compression.
- Delta also supports table partitions just like Hive; however, in this example we are not partitioning the table. In practice, it's recommended to partition on a few select columns to improve performance when there are a large number of rows. Partition on columns that have low cardinality. 
- In the table creation syntax (DDL), notice the format 'USING DELTA'
- Optionally add constraints
   - Insertion will fail if condition check fails

-- Replace the parameter "your-username" with your unique username

In [0]:
%sql
DROP TABLE IF EXISTS odl_user_768559.clickstream_data;
CREATE TABLE odl_user_768559.clickstream_data (
  domain_name STRING,
  event_time STRING,
  user_id STRING,
  country STRING, 
  division STRING,
  page_url STRING,
  page_app STRING,
  page_id STRING,
  browser STRING,
  os STRING,
  platform STRING)
USING DELTA
LOCATION '/tmp/odl_user_768559/delta/clickstream';

ALTER TABLE odl_user_768559.clickstream_data ADD CONSTRAINT dateWithinRange CHECK (event_time > '2015-01-01');
ALTER TABLE odl_user_768559.clickstream_data ADD CONSTRAINT validCountryCode CHECK (country in ('MX','TW','PE','CO','UY','PR','CL','ID','CA','GB','JP','US','PA'));
ALTER TABLE odl_user_768559.clickstream_data DROP CONSTRAINT dateWithinRange;
ALTER TABLE odl_user_768559.clickstream_data DROP CONSTRAINT validCountryCode;

### We will parse a few fields from JSON data we have inside Google Cloud storage and save to the table we created above
- We will read the source data into a Spark DataFrame and then save to the location for the table we created above
- Write the format of the DataFrame as 'delta'
- The write we can be an OVERWRITE or APPEND
   - OVERWRITE will replace all the existing data in the directory with new files
   - APPEND will add new files to the existing set of files
- The schema of our DataFrame must match the schema of the table, otherwise the job will fail. This is Schema Enforcement.

-- Replace the parameter "your-username" with your unique username

In [0]:
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import col

df = sqlContext.read.json("gs://databricksgcplabs/clickstream-json")

parsed_fields = df.select( \
  col('http_vhost_name').alias('domain_name') \
 ,regexp_replace(regexp_replace("event_time","T"," "),"Z","").alias("event_time") \
 ,col('user.browserId').alias('user_id') \
 ,col('page.country').alias('country') \
 ,col('page.division').alias('division') \
 ,col('page.url').alias('page_url') \
 ,col('page.app').alias('page_app') \
 ,col('page.id').alias('page_id') \
 ,col('user.browser').alias('browser') \
 ,col('user.os').alias('os') \
 ,col('user.platform').alias('platform'))

output = (
           parsed_fields.write.format("delta")  # Specify the format as Delta
             .mode("overwrite")  # Specify the mode, OVERWRITE or APPEND. Overwrite will rewrite whereas append will add new records
             .saveAsTable("odl_user_768559.clickstream_data") ## Specify table name or specify a file path if no table is created and you want to write to any directory
         )

### Indexes to improve read performance
- By default, Delta will automatically create Data Skipping indexes for the first 32 columns
- With Data Skipping Indexes, Delta keeps metadata for each Parquet file with the MIN and MAX values for each column in that Parquet file
- This improves query read performance as Delta knows exactly which Parquet files contain the data to satisfy the query and doesn't have to scan all the Parquet files
- Learn more about Data Skipping Indexes: https://docs.gcp.databricks.com/spark/latest/spark-sql/dataskipping-index.html
- You can also optionally create bloom filters: https://docs.gcp.databricks.com/delta/optimizations/bloom-filters.html

-- Replace the parameter "your-username" with your unique username

In [0]:
%sql
SELECT * FROM odl_user_768559.clickstream_data where country = 'US'

domain_name,event_time,user_id,country,division,page_url,page_app,page_id,browser,os,platform
www.initech.com,2016-03-21 13:58:35,95328873-8a81-0f3a-195c-f9159fd182b1,US,minneapolis-stpaul,https://www.initech.com/deals/photoaffections-11-minneapolis-stpaul,deal,420e17aa-8ca3-496a-af80-14e81ad71681-1458568714918-TH0,Firefox,Windows,desktop
www.initech.com,2016-03-21 13:58:35,35a67654-9a5a-68a4-634a-df33273fae55,US,boston,https://www.initech.com/getaways,getaways_itier,385b5936-4739-424e-ac11-122216dbde70-1458568714917-TH0,Chrome,Windows,desktop
www.initech.com,2016-03-21 13:58:35,786b9427-bef2-2258-aac4-f8be48c573d4,US,san-diego,https://www.initech.com/deals/davey-s-locker-sportfishing-36,deal,ddbb8d10-b66b-432d-9c3c-1d9d19e590bd-1458568714723-TH0,Chrome,Windows,desktop
www.initech.com,2016-03-21 13:58:35,86b0ad71-a364-b0fd-d9e7-7782ab503a10,US,null,https://www.initech.com/coupons/stores/asos.com,coupon_itier,6b74e324-cc25-49ed-b143-699b2e3626f7-1458568712712-TH0,Chrome Mobile,Android,touch
www.initech.com,2016-03-21 13:58:35,cc458f4f-7add-4d31-6cc7-3a3f79e94cbd,US,new-york,"https://www.initech.com/browse/los-angeles?address=Los+Angeles,+CA&lat=34.052235&lng=-118.24368&query=braces",pull,64bbf866-780f-43a1-8bf0-12ebadc27b01-1458568714453-TH0,Chrome Mobile,Android,touch
www.initech.com,2016-03-21 13:58:35,340e4d16-0521-afbe-c99a-6c861afdce7f,US,north-jersey,https://www.initech.com/deals/fig-and-olive?p=1&utm_source=newsletter_im&utm_medium=email&division=north-jersey&user=c97fc82d318f6e8a6eed0f470603ea8cbbaf015c6544a4541b5f5f500b1c7e9a&date=20160321&uu=ec530dd6-1ed9-11e2-9ee5-00259060ae08&s=body&c=button&d=deal-page&utm_campaign=ddceb513-a66e-40a2-b84a-450bbd6d313c_0_20160321_treatment1,deal,60fb7188-f38d-474c-8d55-571fd57d3bca-1458568714308-TH0,Chrome,Windows,desktop
www.initech.com,2016-03-21 13:58:35,3ffa3514-1db9-3852-2a66-bf2da4ec077f,US,null,https://www.initech.com/subscription_center/manage/consumer/caaca280-24b2-11e2-8854-00259069d5fe?cmplistId=channel:occasions&utm_source=channel_occasions_im_Beauty-Week&utm_medium=email&user=7b8def3450b34f02c02421c153f9203c139a6e4e5089bccb8a54ba6dd64939e9&date=20160321&uu=caaca280-24b2-11e2-8854-00259069d5fe&s=footer&c=manage-subscriptions-linkBeauty-Week&d=channel_unsub&utm_campaign=46fcce16-dec2-414c-88c0-c12e3756f2d1_0_20160321,sub_center,0f46e6b2-8fa6-460a-80dc-f53e56c61642-1458568708760-TH0,Mobile Safari,iOS,touch
www.initech.com,2016-03-21 13:58:35,9898eb2a-1ea7-11e2-bd8b-00259060b6b8,US,pittsburgh,https://www.initech.com/deals/gl-pittsburgh-improv-2-6?p=11&utm_source=newsletter_im&utm_medium=email&division=pittsburgh&user=4fc768e2a89f333a8cd48c1fd29ea51e16cba1d0afef5b42b93782d05c805985&date=20160320&uu=d4d482c8-2304-11e2-bc45-00259069d5fe&s=body&c=link&d=deal-page&utm_campaign=a370499d-9e7d-4f74-b209-630f8422c6a5_0_20160320_treatment1,deal,a09c27a4-45a4-4a81-854b-bc18d7174948-1458568713008-TH0,IE,Windows,desktop
www.initech.com,2016-03-21 13:58:35,df6ead83-46cf-1291-b252-c49a23d2d03b,US,chattanooga,https://www.initech.com/browse/chattanooga?b_cookie=df6ead83-46cf-1291-b252-c49a23d2d03b&utm_campaign=UserReferral&utm_medium=email&utm_source=deal_ita&fallback=true&deal_id=gg-pyle-hd-widescreen-projector-6,pull,dbe5b72b-9bec-406e-bb39-66efac748f73-1458568711457-TH0,Mobile Safari,iOS,touch
www.initech.com,2016-03-21 13:58:35,63e2f0e9-8257-41b7-3d61-d446020de369,US,new-orleans,https://www.initech.com/,pull,787c8bf3-8bf7-4e80-acee-ce66ff27cb01-1458568714498-TH0,Chrome,Windows,desktop


## Collect table statistics
- It's important to regularly collect table statistics as new data is loaded into the table so the optimizer can choose the most optimal query plan when users submit queries
- Collect statistics for all the individual columns of the table
- Learn more about collecting table statistics: https://docs.gcp.databricks.com/spark/latest/spark-sql/language-manual/sql-ref-syntax-aux-analyze-table.html

-- Replace the parameter "your-username" with your unique username

In [0]:
%sql
ANALYZE TABLE odl_user_768559.clickstream_data COMPUTE STATISTICS FOR COLUMNS domain_name, event_time, user_id, country, division, page_url, page_app, page_id, browser, os, platform

## Let's examine this Delta table

-- Replace the parameter "your-username" with your unique username

In [0]:
%sql
DESCRIBE FORMATTED odl_user_768559.clickstream_data;

col_name,data_type,comment
domain_name,string,
event_time,string,
user_id,string,
country,string,
division,string,
page_url,string,
page_app,string,
page_id,string,
browser,string,
os,string,


### Let's do a file listing of the directory for this table
- The files are open source Parquet with Snappy compression 
- Notice the Delta log directory _delta_log
   - Maintains transaction history of all the data changes for the table
- This Delta table has quite a few small sized parquet files
- Delta can optimize the table by compacting smaller files into fewer larger ones

-- Replace the parameter "your-username" with your unique username

In [0]:
%fs
ls /tmp/odl_user_768559/delta/clickstream

path,name,size
dbfs:/tmp/odl_user_768559/delta/clickstream/_delta_log/,_delta_log/,0
dbfs:/tmp/odl_user_768559/delta/clickstream/part-00000-6d60ea8c-3995-4f06-bab0-ebda60837b42-c000.snappy.parquet,part-00000-6d60ea8c-3995-4f06-bab0-ebda60837b42-c000.snappy.parquet,1489007
dbfs:/tmp/odl_user_768559/delta/clickstream/part-00001-6d3c2c04-b737-4e88-811c-46ac158c844c-c000.snappy.parquet,part-00001-6d3c2c04-b737-4e88-811c-46ac158c844c-c000.snappy.parquet,1406264
dbfs:/tmp/odl_user_768559/delta/clickstream/part-00002-49519401-dd71-4001-955d-8611cde155ab-c000.snappy.parquet,part-00002-49519401-dd71-4001-955d-8611cde155ab-c000.snappy.parquet,1467224
dbfs:/tmp/odl_user_768559/delta/clickstream/part-00003-bd899783-e147-449b-9b14-ff7bb3434094-c000.snappy.parquet,part-00003-bd899783-e147-449b-9b14-ff7bb3434094-c000.snappy.parquet,1372490
dbfs:/tmp/odl_user_768559/delta/clickstream/part-00004-2132f8c6-e547-4194-acd2-34e0863f02c7-c000.snappy.parquet,part-00004-2132f8c6-e547-4194-acd2-34e0863f02c7-c000.snappy.parquet,1395045
dbfs:/tmp/odl_user_768559/delta/clickstream/part-00005-2e8fe4ad-1b7f-41b7-8161-147faa7e04e2-c000.snappy.parquet,part-00005-2e8fe4ad-1b7f-41b7-8161-147faa7e04e2-c000.snappy.parquet,1327927
dbfs:/tmp/odl_user_768559/delta/clickstream/part-00006-6617c66c-574c-4fbd-9535-a9c9d100b72e-c000.snappy.parquet,part-00006-6617c66c-574c-4fbd-9535-a9c9d100b72e-c000.snappy.parquet,1119546
dbfs:/tmp/odl_user_768559/delta/clickstream/part-00007-27a3eed5-f3fc-4911-ab53-456c252b9795-c000.snappy.parquet,part-00007-27a3eed5-f3fc-4911-ab53-456c252b9795-c000.snappy.parquet,1221181
dbfs:/tmp/odl_user_768559/delta/clickstream/part-00008-79519ddb-aa0c-48aa-8afb-f0a3fb18d659-c000.snappy.parquet,part-00008-79519ddb-aa0c-48aa-8afb-f0a3fb18d659-c000.snappy.parquet,986044


### Too many small files will negatively impact performance
- Delta mitigates this by performing a file compaction to create fewer larger sized parquet files.
- Delta also caches data on the local SSD Drives on the VM's after it has first been read from Cloud Storage.
- Thus, subsequent reads of the data will be fetched locally from the SSD drives on the VMs instead of reading again from Cloud Storage.
- Optimize command will by default try to create a 1 GB file and this setting is configurable
   - There is an optional auto-optimize feauture you can enable
- Learn more about Optimize: https://docs.gcp.databricks.com/spark/latest/spark-sql/language-manual/delta-optimize.html

-- Replace the parameter "your-username" with your unique username

In [0]:
%sql
OPTIMIZE odl_user_768559.clickstream_data;

path,metrics
dbfs:/tmp/odl_user_768559/delta/clickstream,"List(1, 10, List(13681843, 13681843, 1.3681843E7, 1, 13681843), List(829350, 1489007, 1261407.8, 10, 12614078), 0, null, 1, 10, 0, true)"


### After performing the file compaction we now have 1 large parquet file
- Delta created the new parquet file
- Delta will read from this new version of the data
- The old files will still be kept unless you explicitly run a VACUUM command to remove them
- We will use the VACUUM command later


-- Replace the parameter "your-username" with your unique username

In [0]:
%fs
ls /tmp/odl_user_768559/delta/clickstream

path,name,size
dbfs:/tmp/odl_user_768559/delta/clickstream/_delta_log/,_delta_log/,0
dbfs:/tmp/odl_user_768559/delta/clickstream/part-00000-2c51d6f8-5a76-4267-a70d-76d54a58d170-c000.snappy.parquet,part-00000-2c51d6f8-5a76-4267-a70d-76d54a58d170-c000.snappy.parquet,13681843
dbfs:/tmp/odl_user_768559/delta/clickstream/part-00000-6d60ea8c-3995-4f06-bab0-ebda60837b42-c000.snappy.parquet,part-00000-6d60ea8c-3995-4f06-bab0-ebda60837b42-c000.snappy.parquet,1489007
dbfs:/tmp/odl_user_768559/delta/clickstream/part-00001-6d3c2c04-b737-4e88-811c-46ac158c844c-c000.snappy.parquet,part-00001-6d3c2c04-b737-4e88-811c-46ac158c844c-c000.snappy.parquet,1406264
dbfs:/tmp/odl_user_768559/delta/clickstream/part-00002-49519401-dd71-4001-955d-8611cde155ab-c000.snappy.parquet,part-00002-49519401-dd71-4001-955d-8611cde155ab-c000.snappy.parquet,1467224
dbfs:/tmp/odl_user_768559/delta/clickstream/part-00003-bd899783-e147-449b-9b14-ff7bb3434094-c000.snappy.parquet,part-00003-bd899783-e147-449b-9b14-ff7bb3434094-c000.snappy.parquet,1372490
dbfs:/tmp/odl_user_768559/delta/clickstream/part-00004-2132f8c6-e547-4194-acd2-34e0863f02c7-c000.snappy.parquet,part-00004-2132f8c6-e547-4194-acd2-34e0863f02c7-c000.snappy.parquet,1395045
dbfs:/tmp/odl_user_768559/delta/clickstream/part-00005-2e8fe4ad-1b7f-41b7-8161-147faa7e04e2-c000.snappy.parquet,part-00005-2e8fe4ad-1b7f-41b7-8161-147faa7e04e2-c000.snappy.parquet,1327927
dbfs:/tmp/odl_user_768559/delta/clickstream/part-00006-6617c66c-574c-4fbd-9535-a9c9d100b72e-c000.snappy.parquet,part-00006-6617c66c-574c-4fbd-9535-a9c9d100b72e-c000.snappy.parquet,1119546
dbfs:/tmp/odl_user_768559/delta/clickstream/part-00007-27a3eed5-f3fc-4911-ab53-456c252b9795-c000.snappy.parquet,part-00007-27a3eed5-f3fc-4911-ab53-456c252b9795-c000.snappy.parquet,1221181


### ACID Transactions
- Multiple processes and users can access and modify a dataset/table and lets end users see consistent views of the data.
- <b>DELETES/UPDATES/UPSERTS</b>
    - Writers can modify a dataset without interfering with current jobs reading the dataset.

### Delta helps you comply with data privacy and data retention</b>
- Use Delta Update and Merge functionality to modify data in base tables
- Use Delta Delete functionality to delete data in base table
- Use Databricks Table Access Control Lists to control access to Databases, Tables, and Views
   - Restrict access to authorized Users and/or Groups 
   - Learn more about Table Access Control Lists: https://docs.gcp.databricks.com/security/access-control/table-acls/index.html

## Update data
- UPDATE: Update rows in single table

-- Replace the parameter "your-username" with your unique username

In [0]:
%sql
select * from odl_user_768559.clickstream_data

domain_name,event_time,user_id,country,division,page_url,page_app,page_id,browser,os,platform
www.initech.com,2016-03-21 13:58:35,95328873-8a81-0f3a-195c-f9159fd182b1,US,minneapolis-stpaul,https://www.initech.com/deals/photoaffections-11-minneapolis-stpaul,deal,420e17aa-8ca3-496a-af80-14e81ad71681-1458568714918-TH0,Firefox,Windows,desktop
www.initech.com,2016-03-21 13:58:35,35a67654-9a5a-68a4-634a-df33273fae55,US,boston,https://www.initech.com/getaways,getaways_itier,385b5936-4739-424e-ac11-122216dbde70-1458568714917-TH0,Chrome,Windows,desktop
www.initech.com,2016-03-21 13:58:35,786b9427-bef2-2258-aac4-f8be48c573d4,US,san-diego,https://www.initech.com/deals/davey-s-locker-sportfishing-36,deal,ddbb8d10-b66b-432d-9c3c-1d9d19e590bd-1458568714723-TH0,Chrome,Windows,desktop
www.initech.cl,2016-03-21 13:58:35,c27c7b79-5afc-4e09-bf30-c7d77519dc34,CL,valdivia,https://www.initech.cl/deals/25990-en-vez-de-40500-por-panales-pampers-juegos-y-suenos-en-talla-a-eleccion-incluye-despacho-k000i0awn10086k20c-1?p=1&nlp=&CID=CL_CRM_1_0_0_81&utm_source=channel_goods&utm_medium=email&sid=e7d62db5-7710-442b-b961-3e081a0a96e9&division=CL_shopping&uh=YzNzenRoZXdsdEpCbEFlVkZMTGI5VVZZNGFKbHpnbHZxdXh1TCtTWk85WT0&eh=d8fa80a44413346aac98354bd25b0d5ef7b5b422f74a6db136e5eb96adaf454b&date=20162103&sender=rm&s=body&c=image&d=deal-page&utm_campaign=1003510511,deal,462fb916-bebe-4bc7-9ec0-0fda48cd426f-1458568713665-TH0,Chrome,Windows,desktop
www.initech.com,2016-03-21 13:58:35,bdd24b24-dc85-083d-3d9d-1d74ec4e37a7,USA,orlando,https://www.initech.com/subscriptions?utm_campaign=UserReferral_mih&utm_source=uu3818076&utm_medium=sms&return_to=/deals/kings-bowl-1?utm_campaign=UserReferral_mih&utm_source=uu3818076&utm_medium=sms&bypass=true,null,6b5eac0c-1b23-4e8d-9b4f-277e0721976e-1458568712497-TH0,Mobile Safari,iOS,touch
www.initech.com,2016-03-21 13:58:35,86b0ad71-a364-b0fd-d9e7-7782ab503a10,US,null,https://www.initech.com/coupons/stores/asos.com,coupon_itier,6b74e324-cc25-49ed-b143-699b2e3626f7-1458568712712-TH0,Chrome Mobile,Android,touch
www.initech.com,2016-03-21 13:58:35,cc458f4f-7add-4d31-6cc7-3a3f79e94cbd,US,new-york,"https://www.initech.com/browse/los-angeles?address=Los+Angeles,+CA&lat=34.052235&lng=-118.24368&query=braces",pull,64bbf866-780f-43a1-8bf0-12ebadc27b01-1458568714453-TH0,Chrome Mobile,Android,touch
www.initech.com,2016-03-21 13:58:35,340e4d16-0521-afbe-c99a-6c861afdce7f,US,north-jersey,https://www.initech.com/deals/fig-and-olive?p=1&utm_source=newsletter_im&utm_medium=email&division=north-jersey&user=c97fc82d318f6e8a6eed0f470603ea8cbbaf015c6544a4541b5f5f500b1c7e9a&date=20160321&uu=ec530dd6-1ed9-11e2-9ee5-00259060ae08&s=body&c=button&d=deal-page&utm_campaign=ddceb513-a66e-40a2-b84a-450bbd6d313c_0_20160321_treatment1,deal,60fb7188-f38d-474c-8d55-571fd57d3bca-1458568714308-TH0,Chrome,Windows,desktop
www.initech.cl,2016-03-21 13:58:35,e252f043-df58-46e4-9481-6fc426e63f37,CL,valdivia,"https://www.initech.cl/browse/valdivia?lat=-40.5761897&lng=-73.1149469&locality=Osorno&administrative_area=X+Regi%C3%B3n&address=Osorno,+X+Regi%C3%B3n,+Chile&locale=es_CL",pull,5c26f1ef-b02d-4c86-b92d-21755705d259-1458568710708-TH0,Chrome,Windows,desktop
www.initech.com,2016-03-21 13:58:35,3ffa3514-1db9-3852-2a66-bf2da4ec077f,US,null,https://www.initech.com/subscription_center/manage/consumer/caaca280-24b2-11e2-8854-00259069d5fe?cmplistId=channel:occasions&utm_source=channel_occasions_im_Beauty-Week&utm_medium=email&user=7b8def3450b34f02c02421c153f9203c139a6e4e5089bccb8a54ba6dd64939e9&date=20160321&uu=caaca280-24b2-11e2-8854-00259069d5fe&s=footer&c=manage-subscriptions-linkBeauty-Week&d=channel_unsub&utm_campaign=46fcce16-dec2-414c-88c0-c12e3756f2d1_0_20160321,sub_center,0f46e6b2-8fa6-460a-80dc-f53e56c61642-1458568708760-TH0,Mobile Safari,iOS,touch


- Let's update event_time to a static value and set domain_name to a static value
- Learn more about the UPDATE command: https://docs.gcp.databricks.com/spark/latest/spark-sql/language-manual/delta-update.html

-- Replace the parameter "your-username" with your unique username

In [0]:
%sql
UPDATE odl_user_768559.clickstream_data SET event_time = '2022-01-01 14:11:32', domain_name= "TEST UPDATE DOMAIN NAME";

num_affected_rows
100000


- The data has been updated

-- Replace the parameter "your-username" with your unique username

In [0]:
%sql
SELECT * FROM odl_user_768559.clickstream_data;

domain_name,event_time,user_id,country,division,page_url,page_app,page_id,browser,os,platform
TEST UPDATE DOMAIN NAME,2022-01-01 14:11:32,95328873-8a81-0f3a-195c-f9159fd182b1,US,minneapolis-stpaul,https://www.initech.com/deals/photoaffections-11-minneapolis-stpaul,deal,420e17aa-8ca3-496a-af80-14e81ad71681-1458568714918-TH0,Firefox,Windows,desktop
TEST UPDATE DOMAIN NAME,2022-01-01 14:11:32,35a67654-9a5a-68a4-634a-df33273fae55,US,boston,https://www.initech.com/getaways,getaways_itier,385b5936-4739-424e-ac11-122216dbde70-1458568714917-TH0,Chrome,Windows,desktop
TEST UPDATE DOMAIN NAME,2022-01-01 14:11:32,786b9427-bef2-2258-aac4-f8be48c573d4,US,san-diego,https://www.initech.com/deals/davey-s-locker-sportfishing-36,deal,ddbb8d10-b66b-432d-9c3c-1d9d19e590bd-1458568714723-TH0,Chrome,Windows,desktop
TEST UPDATE DOMAIN NAME,2022-01-01 14:11:32,c27c7b79-5afc-4e09-bf30-c7d77519dc34,CL,valdivia,https://www.initech.cl/deals/25990-en-vez-de-40500-por-panales-pampers-juegos-y-suenos-en-talla-a-eleccion-incluye-despacho-k000i0awn10086k20c-1?p=1&nlp=&CID=CL_CRM_1_0_0_81&utm_source=channel_goods&utm_medium=email&sid=e7d62db5-7710-442b-b961-3e081a0a96e9&division=CL_shopping&uh=YzNzenRoZXdsdEpCbEFlVkZMTGI5VVZZNGFKbHpnbHZxdXh1TCtTWk85WT0&eh=d8fa80a44413346aac98354bd25b0d5ef7b5b422f74a6db136e5eb96adaf454b&date=20162103&sender=rm&s=body&c=image&d=deal-page&utm_campaign=1003510511,deal,462fb916-bebe-4bc7-9ec0-0fda48cd426f-1458568713665-TH0,Chrome,Windows,desktop
TEST UPDATE DOMAIN NAME,2022-01-01 14:11:32,bdd24b24-dc85-083d-3d9d-1d74ec4e37a7,USA,orlando,https://www.initech.com/subscriptions?utm_campaign=UserReferral_mih&utm_source=uu3818076&utm_medium=sms&return_to=/deals/kings-bowl-1?utm_campaign=UserReferral_mih&utm_source=uu3818076&utm_medium=sms&bypass=true,null,6b5eac0c-1b23-4e8d-9b4f-277e0721976e-1458568712497-TH0,Mobile Safari,iOS,touch
TEST UPDATE DOMAIN NAME,2022-01-01 14:11:32,86b0ad71-a364-b0fd-d9e7-7782ab503a10,US,null,https://www.initech.com/coupons/stores/asos.com,coupon_itier,6b74e324-cc25-49ed-b143-699b2e3626f7-1458568712712-TH0,Chrome Mobile,Android,touch
TEST UPDATE DOMAIN NAME,2022-01-01 14:11:32,cc458f4f-7add-4d31-6cc7-3a3f79e94cbd,US,new-york,"https://www.initech.com/browse/los-angeles?address=Los+Angeles,+CA&lat=34.052235&lng=-118.24368&query=braces",pull,64bbf866-780f-43a1-8bf0-12ebadc27b01-1458568714453-TH0,Chrome Mobile,Android,touch
TEST UPDATE DOMAIN NAME,2022-01-01 14:11:32,340e4d16-0521-afbe-c99a-6c861afdce7f,US,north-jersey,https://www.initech.com/deals/fig-and-olive?p=1&utm_source=newsletter_im&utm_medium=email&division=north-jersey&user=c97fc82d318f6e8a6eed0f470603ea8cbbaf015c6544a4541b5f5f500b1c7e9a&date=20160321&uu=ec530dd6-1ed9-11e2-9ee5-00259060ae08&s=body&c=button&d=deal-page&utm_campaign=ddceb513-a66e-40a2-b84a-450bbd6d313c_0_20160321_treatment1,deal,60fb7188-f38d-474c-8d55-571fd57d3bca-1458568714308-TH0,Chrome,Windows,desktop
TEST UPDATE DOMAIN NAME,2022-01-01 14:11:32,e252f043-df58-46e4-9481-6fc426e63f37,CL,valdivia,"https://www.initech.cl/browse/valdivia?lat=-40.5761897&lng=-73.1149469&locality=Osorno&administrative_area=X+Regi%C3%B3n&address=Osorno,+X+Regi%C3%B3n,+Chile&locale=es_CL",pull,5c26f1ef-b02d-4c86-b92d-21755705d259-1458568710708-TH0,Chrome,Windows,desktop
TEST UPDATE DOMAIN NAME,2022-01-01 14:11:32,3ffa3514-1db9-3852-2a66-bf2da4ec077f,US,null,https://www.initech.com/subscription_center/manage/consumer/caaca280-24b2-11e2-8854-00259069d5fe?cmplistId=channel:occasions&utm_source=channel_occasions_im_Beauty-Week&utm_medium=email&user=7b8def3450b34f02c02421c153f9203c139a6e4e5089bccb8a54ba6dd64939e9&date=20160321&uu=caaca280-24b2-11e2-8854-00259069d5fe&s=footer&c=manage-subscriptions-linkBeauty-Week&d=channel_unsub&utm_campaign=46fcce16-dec2-414c-88c0-c12e3756f2d1_0_20160321,sub_center,0f46e6b2-8fa6-460a-80dc-f53e56c61642-1458568708760-TH0,Mobile Safari,iOS,touch


### Merge data
- MERGE: 
   - Insert or Update data in a table with values from a second table by joining them on 1 or more common keys. 
   - Use cases include for updating Slowly Changing Dimensions (SCD) and Change Data Capture (CDC).
- Learn more about the MERGE command: https://docs.gcp.databricks.com/spark/latest/spark-sql/language-manual/delta-merge-into.html

### Basic CDC Example 
   - ETL process will get new data from source systems
   - This data then needs to be loaded into our target table
   - Load can be INSERT or UPDATE: INSERT new records or UPDATE existing records
   - MERGE command can perform this

- Let's load the new data into a DataFrame which contains new records to INSERT and existing records to UPDATE (Upsert)
- Create a temporary table for this data

In [0]:
df = spark.read.parquet("gs://databricksgcplabs/merge-data/merge.snappy.parquet")
df.createOrReplaceTempView("clickstream_data_stage")

- Let's view the data

In [0]:
%sql
SELECT * FROM clickstream_data_stage

domain_name,event_time,user_id,country,division,page_url,page_app,page_id,browser,os,platform
www.initech.com,2016-03-21 13:04:24,100,JP,hyogo-8,http://www.initech.jp/cid/298913/?p=1&pt=DL01-5555_20160313_28&nlp=&CID=JP_CRM_1_0_0_73&utm_source=newsletter&utm_medium=email&sid=98533a5d-511e-4e4e-a063-f901fad8d79f&division=hyogo&uh=3b5151b88459b4fd88f24c29f300b4f58e14410511172962&eh=15b20e3ca758dca28dc878da357674928a21eb647eca1de9ff6ba56b7df9bd4f&date=20161303&sender=rm&s=body&c=button&d=deal-page&utm_campaign=rm_hyogo_20160313_298913&qaf=NEWS-5555_20160313_28&mid=264983&umm=11172962,initech-japan-pc,00c18604-527b-da36-2b37-df3187a233a2-1458565450551-TH0,IE,Windows,desktop
www.initech.com,2016-03-21 13:04:24,200,US,louisville,https://www.initech.com/goods?utm_source=newsletter_im&utm_medium=email&division=louisville&user=d0133d2555cdc3bb6ba6337fffdfd56d06748b90a08524b40d07f65a85731dae&date=20160321&uu=5c3a2766-eaef-11e1-a9d5-0025906127fe&s=email_goods_widget&c=button&d=all-deals-page&utm_campaign=4d811621-e2c4-4a66-a269-a3d582a26384_0_20160321_treatment2,goods-homepage,34d6d3bd-e61b-42c7-9e6a-38137651fcd9-1458565461995-TH0,Firefox,Windows,desktop
www.initech.com,2016-03-21 13:01:51,200,US,louisville,https://www.initech.com/deals/ga-elk-springs-resort-9?p=7&utm_source=initech_getaways_im&utm_medium=email&division=louisville&user=a2fdc2c3d6f8413861c78928c51a89b59576b7f25e44b638a236a05e5d4e5adf&date=20160320&uu=e26d66de-a634-11e3-8986-00259061289e&s=body&c=image&d=getaways_deal&utm_campaign=5d24b746-0b80-40c5-be47-068635b4e943_0_20160320,deal,34d6d3bd-e61b-42c7-9e6a-38137651fcd9-1458565310440-TH0,Firefox,Windows,desktop
www.initech.com,2016-03-21 13:02:02,200,US,louisville,https://www.initech.com/deals/ga-gate-1-travel-france-9?p=6&utm_source=initech_getaways_im&utm_medium=email&division=louisville&user=a2fdc2c3d6f8413861c78928c51a89b59576b7f25e44b638a236a05e5d4e5adf&date=20160320&uu=e26d66de-a634-11e3-8986-00259061289e&s=body&c=image&d=getaways_deal&utm_campaign=5d24b746-0b80-40c5-be47-068635b4e943_0_20160320,deal,34d6d3bd-e61b-42c7-9e6a-38137651fcd9-1458565321529-TH0,Firefox,Windows,desktop
www.initech.com,2016-03-21 13:03:33,200,US,louisville,https://www.initech.com/deals/drybar-louisville?p=3&utm_source=newsletter_im&utm_medium=email&division=louisville&user=d0133d2555cdc3bb6ba6337fffdfd56d06748b90a08524b40d07f65a85731dae&date=20160321&uu=5c3a2766-eaef-11e1-a9d5-0025906127fe&s=body&c=image&d=deal-page&utm_campaign=4d811621-e2c4-4a66-a269-a3d582a26384_0_20160321_treatment2,deal,34d6d3bd-e61b-42c7-9e6a-38137651fcd9-1458565412447-TH0,Firefox,Windows,desktop
www.initech.com,2016-03-21 13:04:00,200,US,louisville,https://www.initech.com/deals/rocksport?p=4&utm_source=newsletter_im&utm_medium=email&division=louisville&user=d0133d2555cdc3bb6ba6337fffdfd56d06748b90a08524b40d07f65a85731dae&date=20160321&uu=5c3a2766-eaef-11e1-a9d5-0025906127fe&s=body&c=image&d=deal-page&utm_campaign=4d811621-e2c4-4a66-a269-a3d582a26384_0_20160321_treatment2,deal,34d6d3bd-e61b-42c7-9e6a-38137651fcd9-1458565439794-TH0,Firefox,Windows,desktop
www.initech.com,2016-03-21 13:02:17,200,US,louisville,https://www.initech.com/deals/ga-gate-1-travel-france-9?p=6&utm_source=initech_getaways_im&utm_medium=email&division=louisville&user=a2fdc2c3d6f8413861c78928c51a89b59576b7f25e44b638a236a05e5d4e5adf&date=20160320&uu=e26d66de-a634-11e3-8986-00259061289e&s=body&c=image&d=getaways_deal&utm_campaign=5d24b746-0b80-40c5-be47-068635b4e943_0_20160320,deal,34d6d3bd-e61b-42c7-9e6a-38137651fcd9-1458565336435-TH0,Firefox,Windows,desktop
www.initech.com,2016-03-21 13:02:26,200,US,louisville,https://www.initech.com/deals/ga-bk-glenstone-lodge-13?p=3&mobile=false&utm_source=initech_getaways_im&utm_medium=email&division=louisville&user=a2fdc2c3d6f8413861c78928c51a89b59576b7f25e44b638a236a05e5d4e5adf&date=20160320&uu=e26d66de-a634-11e3-8986-00259061289e&s=body&c=image&d=getaways_deal&utm_campaign=5d24b746-0b80-40c5-be47-068635b4e943_0_20160320,deal,34d6d3bd-e61b-42c7-9e6a-38137651fcd9-

- Let's get a count of records from our original table before we run the MERGE operation

-- Replace the parameter "your-username" with your unique username

In [0]:
%sql
SELECT COUNT(1) FROM odl_user_768559.clickstream_data;

count(1)
100000


- Run the MERGE operation to INSERT new data and UPDATE existing data

-- Replace the parameter "your-username" with your unique username

In [0]:
%sql
MERGE INTO odl_user_768559.clickstream_data a /** This is the target table **/
USING clickstream_data_stage b /** This is the source table **/
ON a.user_id = b.user_id AND a.event_time = b.event_time /** This is the join condition **/
WHEN MATCHED THEN /** Update the following columns when matched **/
  UPDATE SET
  a.domain_name = b.domain_name
  ,a.event_time = b.event_time
  ,a.country = b.country
  ,a.division = b.division
  ,a.page_url = b.page_url
  ,a.page_app = b.page_app
  ,a.page_id = b.page_id
  ,a.browser = b.browser
  ,a.os = b.os
  ,a.platform = b.platform
WHEN NOT MATCHED THEN /** Insert a new row when not matched **/
  INSERT (domain_name,event_time,user_id,country,division,page_url,page_app,page_id,browser,os,platform)
  VALUES (b.domain_name,b.event_time,b.user_id,b.country,b.division,b.page_url,b.page_app,b.page_id,b.browser,b.os,b.platform)

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
18,0,0,18


- Let's get a count of records
- Notice before we had 100,000 records and now we have 100,018
- Some existing records were updated and new records were inserted

-- Replace the parameter "your-username" with your unique username

In [0]:
%sql
SELECT COUNT(1) FROM odl_user_768559.clickstream_data

count(1)
100018


## Delete 
- DELETE: Delete rows in single table
- Can also specify conditions in the WHERE clause
- The command below will run 2 SQL statements: First the delete command and then a select command
- Notice that the data has been removed

-- Replace the parameter "your-username" with your unique username

In [0]:
%sql
DELETE FROM odl_user_768559.clickstream_data;
SELECT * FROM odl_user_768559.clickstream_data 

domain_name,event_time,user_id,country,division,page_url,page_app,page_id,browser,os,platform


### We just deleted all of our data! Not to worry, Delta allows us to restore the data

### Data Provenance: History of changes to data
- We can see all the changes made to the table: when change was made, by who, the operation performed and other information
- These are all the different versions of the table over time

-- Replace the parameter "your-username" with your unique username

In [0]:
%sql
DESCRIBE HISTORY odl_user_768559.clickstream_data;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
9,2022-10-19T17:44:30.370+0000,3824250108042418,odl_user_768559@databrickslabs.com,DELETE,Map(predicate -> []),null,List(3714474033348371),1019-150532-rhiw2tml,8,WriteSerializable,false,"Map(numRemovedFiles -> 2, numAddedChangeFiles -> 0, executionTimeMs -> 21, scanTimeMs -> 20, rewriteTimeMs -> 0)",null
8,2022-10-19T17:40:45.229+0000,3824250108042418,odl_user_768559@databrickslabs.com,MERGE,"Map(predicate -> ((a.user_id = b.user_id) AND (a.event_time = b.event_time)), matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(3714474033348371),1019-150532-rhiw2tml,7,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 1, executionTimeMs -> 2083, numTargetRowsInserted -> 18, scanTimeMs -> 1252, numTargetRowsUpdated -> 0, numOutputRows -> 18, numTargetChangeFilesAdded -> 0, numSourceRows -> 18, numTargetFilesRemoved -> 0, rewriteTimeMs -> 786)",null
7,2022-10-19T17:39:32.342+0000,3824250108042418,odl_user_768559@databrickslabs.com,UPDATE,Map(),null,List(3714474033348371),1019-150532-rhiw2tml,6,WriteSerializable,false,"Map(numRemovedFiles -> 1, numCopiedRows -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 1818, scanTimeMs -> 39, numAddedFiles -> 1, numUpdatedRows -> 100000, rewriteTimeMs -> 1779)",null
6,2022-10-19T17:36:00.659+0000,3824250108042418,odl_user_768559@databrickslabs.com,OPTIMIZE,"Map(predicate -> [], zOrderBy -> [], batchId -> 0, auto -> false)",null,List(3714474033348371),1019-150532-rhiw2tml,5,SnapshotIsolation,false,"Map(numRemovedFiles -> 10, numRemovedBytes -> 12614078, p25FileSize -> 13681843, minFileSize -> 13681843, numAddedFiles -> 1, maxFileSize -> 13681843, p75FileSize -> 13681843, p50FileSize -> 13681843, numAddedBytes -> 13681843)",null
5,2022-10-19T17:27:24.889+0000,3824250108042418,odl_user_768559@databrickslabs.com,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> false, description -> null, partitionBy -> [], properties -> {})",null,List(3714474033348371),1019-150532-rhiw2tml,4,WriteSerializable,false,"Map(numFiles -> 10, numOutputRows -> 100000, numOutputBytes -> 12614078)",null
4,2022-10-19T17:26:42.264+0000,3824250108042418,odl_user_768559@databrickslabs.com,DROP CONSTRAINT,"Map(name -> validCountryCode, expr -> country in ( 'MX' , 'TW' , 'PE' , 'CO' , 'UY' , 'PR' , 'CL' , 'ID' , 'CA' , 'GB' , 'JP' , 'US' , 'PA' ), existed -> true)",null,List(3714474033348371),1019-150532-rhiw2tml,3,SnapshotIsolation,true,Map(),null
3,2022-10-19T17:26:40.814+0000,3824250108042418,odl_user_768559@databrickslabs.com,DROP CONSTRAINT,"Map(name -> dateWithinRange, expr -> event_time > '2015-01-01', existed -> true)",null,List(3714474033348371),1019-150532-rhiw2tml,2,SnapshotIsolation,true,Map(),null
2,2022-10-19T17:26:39.425+0000,3824250108042418,odl_user_768559@databrickslabs.com,ADD CONSTRAINT,"Map(name -> validCountryCode, expr -> country in ( 'MX' , 'TW' , 'PE' , 'CO' , 'UY' , 'PR' , 'CL' , 'ID' , 'CA' , 'GB' , 'JP' , 'US' , 'PA' ))",null,List(3714474033348371),1019-150532-rhiw2tml,1,SnapshotIsolation,false,Map(),null
1,2022-10-19T17:26:37.920+0000,3824250108042418,odl_user_768559@databrickslabs.com,ADD CONSTRAINT,"Map(name -> dateWithinRange, expr -> event_time > '2015-01-01')",null,List(3714474033348371),1019-150532-rhiw2tml,0,SnapshotIsolation,false,Map(),null
0,2022-10-19T17:26:35.910+0000,3824250108042418,odl_user_768559@databrickslabs.com,CREATE TABLE,"Map(isManaged -> false, description -> null, partitionBy -> [], properties -> {})",null,List(3714474033348371),1019-150532-rhiw2tml,null,SnapshotIsolation,true,Map(),null


### We can rollback to an earlier version if we have not removed (VACUUM) the old files
- We made several changes to the data: UPDATE, MERGE, AND DELETE
- Delta versions the Parquet files and does not delete them
- Old file are kept unless you remove them using the VACUUM command
- Since the old files are still there we can perform Time Travel

-- Replace the parameter "your-username" with your unique username

In [0]:
%fs
ls /tmp/odl_user_768559/delta/clickstream

path,name,size
dbfs:/tmp/odl_user_768559/delta/clickstream/_delta_log/,_delta_log/,0
dbfs:/tmp/odl_user_768559/delta/clickstream/part-00000-201d733c-cd5e-4179-8036-c2be55443fd3-c000.snappy.parquet,part-00000-201d733c-cd5e-4179-8036-c2be55443fd3-c000.snappy.parquet,13658566
dbfs:/tmp/odl_user_768559/delta/clickstream/part-00000-2c51d6f8-5a76-4267-a70d-76d54a58d170-c000.snappy.parquet,part-00000-2c51d6f8-5a76-4267-a70d-76d54a58d170-c000.snappy.parquet,13681843
dbfs:/tmp/odl_user_768559/delta/clickstream/part-00000-6d60ea8c-3995-4f06-bab0-ebda60837b42-c000.snappy.parquet,part-00000-6d60ea8c-3995-4f06-bab0-ebda60837b42-c000.snappy.parquet,1489007
dbfs:/tmp/odl_user_768559/delta/clickstream/part-00000-db62c10e-cd81-4a5e-b0ef-5e908d5c62c9-c000.snappy.parquet,part-00000-db62c10e-cd81-4a5e-b0ef-5e908d5c62c9-c000.snappy.parquet,6315
dbfs:/tmp/odl_user_768559/delta/clickstream/part-00001-6d3c2c04-b737-4e88-811c-46ac158c844c-c000.snappy.parquet,part-00001-6d3c2c04-b737-4e88-811c-46ac158c844c-c000.snappy.parquet,1406264
dbfs:/tmp/odl_user_768559/delta/clickstream/part-00002-49519401-dd71-4001-955d-8611cde155ab-c000.snappy.parquet,part-00002-49519401-dd71-4001-955d-8611cde155ab-c000.snappy.parquet,1467224
dbfs:/tmp/odl_user_768559/delta/clickstream/part-00003-bd899783-e147-449b-9b14-ff7bb3434094-c000.snappy.parquet,part-00003-bd899783-e147-449b-9b14-ff7bb3434094-c000.snappy.parquet,1372490
dbfs:/tmp/odl_user_768559/delta/clickstream/part-00004-2132f8c6-e547-4194-acd2-34e0863f02c7-c000.snappy.parquet,part-00004-2132f8c6-e547-4194-acd2-34e0863f02c7-c000.snappy.parquet,1395045
dbfs:/tmp/odl_user_768559/delta/clickstream/part-00005-2e8fe4ad-1b7f-41b7-8161-147faa7e04e2-c000.snappy.parquet,part-00005-2e8fe4ad-1b7f-41b7-8161-147faa7e04e2-c000.snappy.parquet,1327927


## Time Travel to recover previous version of the data
- Let's restore to a previous version of the table
- You restore by version number or the timestamp

-- Replace the parameter "your-username" with your unique username

In [0]:
%sql
INSERT INTO odl_user_768559.clickstream_data
SELECT * FROM odl_user_768559.clickstream_data VERSION AS OF 5 /** Can also specify the timestamp instead of the version **/

num_affected_rows,num_inserted_rows
100000,100000


- The table has been restored

-- Replace the parameter "your-username" with your unique username

In [0]:
%sql
SELECT * FROM odl_user_768559.clickstream_data

domain_name,event_time,user_id,country,division,page_url,page_app,page_id,browser,os,platform
www.initech.com,2016-03-21 13:58:35,95328873-8a81-0f3a-195c-f9159fd182b1,US,minneapolis-stpaul,https://www.initech.com/deals/photoaffections-11-minneapolis-stpaul,deal,420e17aa-8ca3-496a-af80-14e81ad71681-1458568714918-TH0,Firefox,Windows,desktop
www.initech.com,2016-03-21 13:58:35,35a67654-9a5a-68a4-634a-df33273fae55,US,boston,https://www.initech.com/getaways,getaways_itier,385b5936-4739-424e-ac11-122216dbde70-1458568714917-TH0,Chrome,Windows,desktop
www.initech.com,2016-03-21 13:58:35,786b9427-bef2-2258-aac4-f8be48c573d4,US,san-diego,https://www.initech.com/deals/davey-s-locker-sportfishing-36,deal,ddbb8d10-b66b-432d-9c3c-1d9d19e590bd-1458568714723-TH0,Chrome,Windows,desktop
www.initech.cl,2016-03-21 13:58:35,c27c7b79-5afc-4e09-bf30-c7d77519dc34,CL,valdivia,https://www.initech.cl/deals/25990-en-vez-de-40500-por-panales-pampers-juegos-y-suenos-en-talla-a-eleccion-incluye-despacho-k000i0awn10086k20c-1?p=1&nlp=&CID=CL_CRM_1_0_0_81&utm_source=channel_goods&utm_medium=email&sid=e7d62db5-7710-442b-b961-3e081a0a96e9&division=CL_shopping&uh=YzNzenRoZXdsdEpCbEFlVkZMTGI5VVZZNGFKbHpnbHZxdXh1TCtTWk85WT0&eh=d8fa80a44413346aac98354bd25b0d5ef7b5b422f74a6db136e5eb96adaf454b&date=20162103&sender=rm&s=body&c=image&d=deal-page&utm_campaign=1003510511,deal,462fb916-bebe-4bc7-9ec0-0fda48cd426f-1458568713665-TH0,Chrome,Windows,desktop
www.initech.com,2016-03-21 13:58:35,bdd24b24-dc85-083d-3d9d-1d74ec4e37a7,USA,orlando,https://www.initech.com/subscriptions?utm_campaign=UserReferral_mih&utm_source=uu3818076&utm_medium=sms&return_to=/deals/kings-bowl-1?utm_campaign=UserReferral_mih&utm_source=uu3818076&utm_medium=sms&bypass=true,null,6b5eac0c-1b23-4e8d-9b4f-277e0721976e-1458568712497-TH0,Mobile Safari,iOS,touch
www.initech.com,2016-03-21 13:58:35,86b0ad71-a364-b0fd-d9e7-7782ab503a10,US,null,https://www.initech.com/coupons/stores/asos.com,coupon_itier,6b74e324-cc25-49ed-b143-699b2e3626f7-1458568712712-TH0,Chrome Mobile,Android,touch
www.initech.com,2016-03-21 13:58:35,cc458f4f-7add-4d31-6cc7-3a3f79e94cbd,US,new-york,"https://www.initech.com/browse/los-angeles?address=Los+Angeles,+CA&lat=34.052235&lng=-118.24368&query=braces",pull,64bbf866-780f-43a1-8bf0-12ebadc27b01-1458568714453-TH0,Chrome Mobile,Android,touch
www.initech.com,2016-03-21 13:58:35,340e4d16-0521-afbe-c99a-6c861afdce7f,US,north-jersey,https://www.initech.com/deals/fig-and-olive?p=1&utm_source=newsletter_im&utm_medium=email&division=north-jersey&user=c97fc82d318f6e8a6eed0f470603ea8cbbaf015c6544a4541b5f5f500b1c7e9a&date=20160321&uu=ec530dd6-1ed9-11e2-9ee5-00259060ae08&s=body&c=button&d=deal-page&utm_campaign=ddceb513-a66e-40a2-b84a-450bbd6d313c_0_20160321_treatment1,deal,60fb7188-f38d-474c-8d55-571fd57d3bca-1458568714308-TH0,Chrome,Windows,desktop
www.initech.cl,2016-03-21 13:58:35,e252f043-df58-46e4-9481-6fc426e63f37,CL,valdivia,"https://www.initech.cl/browse/valdivia?lat=-40.5761897&lng=-73.1149469&locality=Osorno&administrative_area=X+Regi%C3%B3n&address=Osorno,+X+Regi%C3%B3n,+Chile&locale=es_CL",pull,5c26f1ef-b02d-4c86-b92d-21755705d259-1458568710708-TH0,Chrome,Windows,desktop
www.initech.com,2016-03-21 13:58:35,3ffa3514-1db9-3852-2a66-bf2da4ec077f,US,null,https://www.initech.com/subscription_center/manage/consumer/caaca280-24b2-11e2-8854-00259069d5fe?cmplistId=channel:occasions&utm_source=channel_occasions_im_Beauty-Week&utm_medium=email&user=7b8def3450b34f02c02421c153f9203c139a6e4e5089bccb8a54ba6dd64939e9&date=20160321&uu=caaca280-24b2-11e2-8854-00259069d5fe&s=footer&c=manage-subscriptions-linkBeauty-Week&d=channel_unsub&utm_campaign=46fcce16-dec2-414c-88c0-c12e3756f2d1_0_20160321,sub_center,0f46e6b2-8fa6-460a-80dc-f53e56c61642-1458568708760-TH0,Mobile Safari,iOS,touch


### VACUUM command to remove old files
- Use the VACUUM commamnd to remove old files
- We do not recommend that you set a retention interval shorter than 7 days, because old snapshots and uncommitted files can still be in use by concurrent readers or writers to the table. 
- If VACUUM cleans up active files, concurrent readers can fail or, worse, tables can be corrupted when VACUUM deletes files that have not yet been committed.
- By default, VACUUM is set to 7 days. You can override this setting by setting spark.databricks.delta.retentionDurationCheck.enabled to false. 

-- Replace the parameter "your-username" with your unique username

- Let's first Optimize the table

In [0]:
%sql
OPTIMIZE odl_user_768559.clickstream_data;

path,metrics
dbfs:/tmp/odl_user_768559/delta/clickstream,"List(1, 8, List(13681843, 13681843, 1.3681843E7, 1, 13681843), List(829350, 2544913, 1606739.5, 8, 12853916), 0, null, 1, 8, 0, true)"


In [0]:
%sql
SET spark.databricks.delta.retentionDurationCheck.enabled=false;
VACUUM odl_user_768559.clickstream_data RETAIN 0 HOURS;

path
dbfs:/tmp/odl_user_768559/delta/clickstream


- Now notice all the old files have been deleted

-- Replace the parameter "your-username" with your unique username

In [0]:
%fs
ls /tmp/odl_user_768559/delta/clickstream

path,name,size
dbfs:/tmp/odl_user_768559/delta/clickstream/_delta_log/,_delta_log/,0
dbfs:/tmp/odl_user_768559/delta/clickstream/part-00000-6caff80e-8d12-4e7b-a3d2-80a8daf5a6de-c000.snappy.parquet,part-00000-6caff80e-8d12-4e7b-a3d2-80a8daf5a6de-c000.snappy.parquet,13681843


#### [Click here to return to agenda]($./Agenda)